In [75]:
from sklearn.datasets import make_blobs
from keras.src.saving import load_model
import pickle
import numpy as np
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler
from templates.FunctionsBuild import create_sequential_model

In [10]:
scalar_path = "../models/scalar.pickle"
path_model = "../models/model.keras"
# load model
model = load_model(path_model)
# open scalar
with open(scalar_path, "rb") as f:
    scalar = pickle.load(f)
# new instances where we do not know the answer
Xnew, _ = make_blobs(n_samples=3, centers=2, n_features=2, random_state=1)
Xnew = scalar.transform(Xnew)
# make a prediction
# ynew = model.predict_step(Xnew)
ynew = model.predict(Xnew)
# show the inputs and predicted outputs
for i in range(len(Xnew)):
	print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
X=[0.89337759 0.65864154], Predicted=[0.17742321]
X=[0.29097707 0.12978982], Predicted=[0.9963976]
X=[0.78082614 0.75391697], Predicted=[0.17742321]


In [2]:
with open("../data/data_train_2024-05-30.pkl", "rb") as f:
    data_train = pickle.load(f)


In [36]:
data_x = data_train["absences"]["x"]
data_y = data_train["absences"]["y"]
print("columns: ", data_train["absences"]["columns"])

data_x = np.array(data_x)
data_y = np.array(data_y)
print(data_x.shape, data_y.shape)
print(data_x[:, 0])
non_zeros = np.count_nonzero(data_y)
print(non_zeros)

columns:  ('id_emp', 'day_of_week', 'month', 'total_value', 'mean', 'std', 'median', 'var')
(107103, 9) (107103, 2)
[12. 12. 12. ... 59. 59. 59.]
158


In [206]:
dict_data = {}
for index, column in enumerate(data_train["absences"]["columns"]):
    dict_data[column] = data_x[:, index]
dict_data["y"] = data_y[:, 0]
dict_data["y_val"] = data_y[:, 1]


In [207]:
df = pd.DataFrame(dict_data)
df_lates = df[df["y"] != 0]
n_absences = len(df_lates)
df_normal = df[df["y"] == 0]
n_normal = len(df_normal)
n_sample_normal = n_absences * 5
df_normal_sample = df_normal.sample(n_sample_normal)
df_final = pd.concat([df_lates, df_normal_sample])
# randomize data
df_final = df_final.sample(frac=1).reset_index(drop=True)
data_train_sample = df_final.values
print(data_train_sample.shape)
x = data_train_sample[:, 1:-2]
y = data_train_sample[:, -2:-1]
print(x.shape)


(474, 10)
(474, 7)


In [208]:
scalar = MinMaxScaler()
scalar.fit(x)
X = scalar.transform(x)
pathfile = "../models/scalar_abseces.pkl"
with open(pathfile, "wb") as f:
    pickle.dump(data_train, f)

In [165]:
scalar_path = "../models/scalar.pkl"
path_model = "../models/model.keras"
# load model
model = load_model(path_model)
# open scalar
with open(scalar_path, "rb") as f:
    scalar = pickle.load(f)

[[0.6        0.72727273 0.         ... 0.         0.         0.        ]
 [0.8        0.72727273 0.         ... 0.         0.         0.        ]
 [0.6        0.81818182 0.         ... 0.         0.         0.        ]
 ...
 [0.2        0.90909091 0.         ... 0.         0.         0.        ]
 [0.4        0.45454545 0.         ... 0.         0.         0.        ]
 [0.2        0.63636364 0.         ... 0.         0.         0.        ]]


In [70]:
# model.add(Dense(4, input_shape=(2,), activation='relu'))
# model.add(Dense(4, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
dict_layers = [
    {"units": 4, "shape": (len(X[0],)), "activation": "relu"},
    {"units": 4, "shape": (len(X[0],)), "activation": "relu"},
    {"units": 1, "shape": (len(X[0],)), "activation": "sigmoid"}
]
filepath = "../models/model_test_absences.keras"
model = create_sequential_model(dict_layers, filepath=filepath)

TypeError: create_sequential_model() got an unexpected keyword argument 'filepath'

In [181]:
# load model
filepath = "../models/model_test_classes.keras"
new_model = keras.models.load_model(filepath)
print(new_model)

<Sequential name=sequential, built=True>


In [182]:
new_model.compile(loss='binary_crossentropy', optimizer='adam')

In [183]:
new_model.fit(X, y, epochs=500, verbose=0)

In [184]:
filepath = "../models/model_test_absences_trained.keras"
new_model.save(filepath)

In [196]:
# new instances where we do not know the answer
users = df["id_emp"].unique()
print(users)
data_out = []
for emp in users:
    df_emp = df[(df["id_emp"] == emp) & (df["y"] == 1)]
    if len(df_emp)>0:
        data_out.append(df_emp.iloc[-1].values)
    else:
        data_out.append([emp,  1,  1,  0, 0,  0,  0, 0,  0,  0])
data_out = np.array(data_out)
pathfile = "../data/last_vals_absences.pkl"
with open(pathfile, "wb") as f:
    pickle.dump(data_out, f)


[12. 18. 13. 45. 57.  4. 58. 51. 24.  6. 37. 23. 14. 27.  8.  1. 15. 10.
 36. 11. 49. 56. 46.  2.  9. 28. 52. 44. 55. 26. 35. 19. 33. 31.  3.  5.
 17. 22. 25. 29. 30. 32. 34. 38. 41. 42. 47. 48. 50. 53. 59.]


In [195]:
# new instances where we do not know the answer
day_of_week = 6
month = 6
rows = data_out.shape[0]
print(rows)
for i in range(rows):
    data_out[i, 1] = day_of_week
    data_out[i, 2] = month
Xnew = data_out[:, 1:-2]
print(Xnew.shape)
Xnew = scalar.transform(Xnew)
ynew = new_model.predict(Xnew)
threshold = 0.97
counter=0
for index, item in enumerate(ynew):
    # print("value: ", item, "data ", data_out[index, :])
    if item>threshold:
        counter +=1
        print(f"El empleado {data_out[index, 0]} es posible que falte el dia {day_of_week} en el mes {month}")
        # print(data_out[index, :])
        
print(f"posibles faltas: {counter}")
# print(data_out[0,:])

51
(51, 7)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
El empleado 13.0 es posible que falte el dia 6 en el mes 6
El empleado 45.0 es posible que falte el dia 6 en el mes 6
El empleado 24.0 es posible que falte el dia 6 en el mes 6
El empleado 36.0 es posible que falte el dia 6 en el mes 6
El empleado 56.0 es posible que falte el dia 6 en el mes 6
El empleado 46.0 es posible que falte el dia 6 en el mes 6
El empleado 9.0 es posible que falte el dia 6 en el mes 6
El empleado 52.0 es posible que falte el dia 6 en el mes 6
posibles faltas: 8


In [ ]:
###################------------------------lates models--------------------------------------######################

In [238]:
type_class = "lates"
data_x = data_train[type_class]["x"]
data_y = data_train[type_class]["y"]
print("columns: ", data_train[type_class]["columns"])
data_x = np.array(data_x)
data_y = np.array(data_y)
print(data_x.shape, data_y.shape)
print(data_x[:, 0])
non_zeros = np.count_nonzero(data_y)
dict_data = {}
for index, column in enumerate(data_train[type_class]["columns"]):
    dict_data[column] = data_x[:, index]
dict_data["y"] = data_y[:, 0]
dict_data["y_val"] = data_y[:, 1]
print(dict_data["y_val"])

columns:  ('id_emp', 'day_of_week', 'month', 'total_value', 'mean', 'std', 'median', 'var')
(107103, 9) (107103, 2)
[12. 12. 12. ... 59. 59. 59.]
[0. 0. 0. ... 0. 0. 0.]


In [251]:
df = pd.DataFrame(dict_data)
df_lates = df[df["y"] != 0]
# print(df_lates.head())
n_lates = len(df_lates)
print(f"n lates: {n_lates}")
df_normal = df[df["y"] == 0]
n_normal = len(df_normal)
n_sample_normal = n_lates * 3
df_normal_sample = df_normal.sample(n_sample_normal)
df_final = pd.concat([df_lates, df_normal_sample])
# randomize data
df_final = df_final.sample(frac=1).reset_index(drop=True)
data_train_sample = df_final.values
print(data_train_sample.shape)
x = data_train_sample[:, 1:-2]
y1 = data_train_sample[:, -2:-1]
y2 = data_train_sample[:, -1] 
print(x.shape)


n lates: 287
(1148, 10)
(1148, 7)


In [222]:
a = np.array([[1, 2, 3],[1, 2, 3], [4,4,4]])
print(a)
new_column = a[:,-1]
print(new_column)
b = np.c_[a, new_column]
print(b)

[[1 2 3]
 [1 2 3]
 [4 4 4]]
[3 3 4]
[[1 2 3 3]
 [1 2 3 3]
 [4 4 4 4]]


In [252]:
scalar_lates = MinMaxScaler()
scalar_lates.fit(x)
X = scalar_lates.transform(x)
pathfile = "../models/scalar_lates.pkl"
with open(pathfile, "wb") as f:
    pickle.dump(data_train, f)

In [ ]:
scalar_path = "../models/scalar_lates.pkl"
path_model = "../models/model.keras"
# load model
model = load_model(path_model)
# open scalar
with open(scalar_path, "rb") as f:
    scalar_lates = pickle.load(f)

In [253]:
# load model classes
filepath = "../models/model_test_classes.keras"
new_model_lates_class = keras.models.load_model(filepath)
new_model_lates_class.compile(loss='binary_crossentropy', optimizer='adam')
new_model_lates_class.fit(X, y1, epochs=500, verbose=0)
filepath = "../models/model_test_lates_trained_class.keras"
new_model_lates_class.save(filepath)
print("output: ", filepath)

output:  ../models/model_test_lates_trained_class.keras


In [243]:
# load model linear
filepath = "../models/model_test_base_linear.keras"
new_model_lates_linear = keras.models.load_model(filepath)
new_model_lates_linear.compile(loss='mse', optimizer='adam')
new_model_lates_linear.fit(X, y2, epochs=500, verbose=0)
filepath = "../models/model_test_lates_trained_linear.keras"
new_model_lates_linear.save(filepath)
print("output: ", filepath)

output:  ../models/model_test_lates_trained_linear.keras


In [254]:
users = df["id_emp"].unique()
print(users)
data_out = []
for emp in users:
    df_emp = df[(df["id_emp"] == emp) & (df["y"] == 1)]
    if len(df_emp) > 0:
        data_out.append(df_emp.iloc[-1].values)
    else:
        data_out.append([emp, 1, 1, 0, 0, 0, 0, 0, 0, 0])
data_out = np.array(data_out)
pathfile = "../data/last_vals_lates.pkl"
with open(pathfile, "wb") as f:
    pickle.dump(data_out, f)


[12. 18. 13. 45. 57.  4. 58. 51. 24.  6. 37. 23. 14. 27.  8.  1. 15. 10.
 36. 11. 49. 56. 46.  2.  9. 28. 52. 44. 55. 26. 35. 19. 33. 31.  3.  5.
 17. 22. 25. 29. 30. 32. 34. 38. 41. 42. 47. 48. 50. 53. 59.]


In [255]:
# new instances where we do not know the answer
day_of_week = 2
month = 6
rows = data_out.shape[0]
print(rows)
for i in range(rows):
    data_out[i, 1] = day_of_week
    data_out[i, 2] = month
Xnew = data_out[:, 1:-2]
print(Xnew.shape)
Xnew = scalar_lates.transform(Xnew)
ynew1 = new_model_lates_class.predict(Xnew)
ynew2 = new_model_lates_linear.predict(Xnew)
print(ynew1)
print(ynew2)
# late days
threshold = 0.5
counter = 0
for index, item in enumerate(ynew1):
    # print("value: ", item, "data ", data_out[index, :])
    if item > threshold:
        counter += 1
        print(f"El empleado {data_out[index, 0]} es posible que se atrase el dia {day_of_week} en el mes {month} una cantidad {ynew2[index]}")
print(f"posibles dias tarde: {counter}")
# print(data_out[0,:])


51
(51, 7)
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[0.0750278 ]
 [0.04069853]
 [0.03164852]
 [0.0825123 ]
 [0.0381073 ]
 [0.01278051]
 [0.01492105]
 [0.01586521]
 [0.03611137]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01877124]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.0214959 ]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.06076619]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.01278051]
 [0.0261017 ]
 [0.01278051]]
[[ 0.43143988]
 [ 0.46435928]
 [ 0.5597615 ]
 [ 0.6039946 ]
 [ 0.28502512]
 [ 0.00287199]
 [ 0.262089  ]
 [ 0.237468  ]
 [ 0.44613576]
 [-0.0245285 ]
 [ 0.03518534]
 [ 0.03518534]
 [ 0.00287199]
 [ 